Определение: Линейное программирование – математическая дисциплина, посвященная теории и методам решения экстремальных задач на множествах n- мерного пространства, задаваемых системами линейными уравнений и неравенств. Общая задача линейного программирования имеет вид:

![](http://habrastorage.org/r/w1560/webt/t2/_r/0i/t2_r0ia0zdnsgafhpquihxldma4.png)

Каноническая форма задачи ЛП:

![](https://habrastorage.org/r/w1560/webt/al/nw/1q/alnw1qyxxinzazwyx8q3wnym9wk.png)

Алгоритм перехода от произвольной задачи ЛП к канонической форме:

1. Неравенства с отрицательными $b_i$ умножаем на (-1).
2. Если неравенство вида (≤), то к левой части добавляем $s_i$ – добавочную переменную, и получаем равенство.
3. Если неравенство вида (≥), то из левой части вычитаем $s_i$, и получаем равенство.

In [40]:
import json
import numpy as np
import math

# Load data from a specified JSON file
def load_json_data(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    return data

# Transform data to canonical form for the simplex algorithm
def transform_to_canonical_form(data):
    # Extract constraints from data
    constraints = data["constraints"]
    # Determine the number of original variables
    num_original_vars = len(data["f"])
    # Determine the number of constraints
    num_constraints = len(constraints)
    
    constraint_matrix = []
    right_hand_side_values = []

    # Convert ≥ constraints to ≤ by multiplying by -1
    for constraint in constraints:
        if constraint["type"] == "gte":
            constraint["coefs"] = [-coef for coef in constraint["coefs"]]
            constraint["b"] = -constraint["b"]
    
    # For inequalities, add slack variables
    for idx, constraint in enumerate(constraints):
        coefs = constraint["coefs"] + [0] * num_constraints
        if constraint["type"] in ["lte", "gte"]:
            coefs[num_original_vars + idx] = 1
        constraint_matrix.append(coefs)
        right_hand_side_values.append(constraint["b"])

    # Remove unnecessary slack variable columns
    for idx in reversed(range(num_original_vars, num_original_vars + num_constraints)):
        if all(row[idx] == 0 for row in constraint_matrix):
            for row in constraint_matrix:
                row.pop(idx)
                
    # If right-hand-side value is negative, multiply its row by -1
    for idx, value in enumerate(right_hand_side_values):
        if value < 0:
            constraint_matrix[idx] = [-coef for coef in constraint_matrix[idx]]
            right_hand_side_values[idx] = -value

    return constraint_matrix, right_hand_side_values

Определение: Точка $Х ∈ D$ называется угловой точкой, если представление$ Х= αХ^1+ (1-α) Х^2,где Х^1,Х^2 ∈D;0< α<1 $ возможно только при $Х^1=Х^2 $.

Иными словами, невозможно найти две точки в области, интервал проходящий через которые содержит $Х$ (т.е. $Х$ – не внутренняя точка).

Графический способ решения задачи ЛП показывает, что нахождение оптимального решения ассоциируется с угловой точкой. Это является основной концепцией при разработке симплекс-метода.

Определение: Пусть есть система m уравнений и n неизвестных (m < n). Разделим переменные на два множества: (n-m) переменные положим равными нулю, а остальные m переменных определяются решением системы исходных уравнений. Если это решение единственно, то тогда ненулевые m переменных называют базисными; нулевые (n-m) переменных – свободными (небазисными), а соответствующие результирующие значения переменных называют базисным решением.

Симплекс-метод позволяет эффективно найти оптимальное решение, избегая простой перебор всех возможных угловых точек. Основной принцип метода: вычисления начинаются с какого-то «стартового» базисного решения, а затем ведется поиск решений, «улучшающих» значение целевой функции. Это возможно только в том случае, если возрастание какой-то переменной приведет к увеличению значения функционала.

Необходимые условия для применения симплекс-метода:

1. Задача должна иметь каноническую форму.
2. У задачи должен быть явно выделенный базис.

Определение: Явно выделенным базисом будем называть вектора вида:$(..0100..)^T, (..010..)^T,(..0010..)^T...$, т.е. только одна координата вектора ненулевая и равна 1.

Замечание: Базисный вектор имеет размерность (m*1), где m – количество уравнений в системе ограничений.

Для удобства вычислений и наглядности обычно пользуются симплекс-таблицами:

![](http://habrastorage.org/r/w1560/webt/nk/pj/jo/nkpjjolxzgspzdiyp1ekj1yxmiq.png)

В первой строке указывают «наименование» всех переменных.
В первом столбце указывают номера базисных переменных, а в последней ячейке – букву Z (это строка функционала).
В «середине таблицы» указывают коэффициенты матрицы ограничений — aij.
Последний столбец – вектор правых частей соответствующих уравнений системы ограничений.
Крайняя правая ячейка – значение целевой функции. На первой итерации ее полагают равной 0.

Замечание: Базис – переменные, коэффициенты в матрице ограничений при которых образуют базисные вектора.

Замечание: Если ограничения в исходной задаче представлены неравенствами вида ≤, то при приведении задачи к канонической форме, введенные дополнительные переменные образуют начальное базисное решение.

Замечание: Коэффициенты в строке функционала берутся со знаком “-”.

In [41]:
# Set up the initial table for the simplex method
def setup_initial_simplex_table(constraint_matrix, right_hand_side_values, objective_coefs, optimization_goal):
    num_constraints = len(right_hand_side_values)
    num_original_vars = len(objective_coefs)

    # Identify basic and non-basic variables
    num_slack_vars = len(constraint_matrix[0]) - num_original_vars
    basic_vars = list(range(num_original_vars, num_original_vars + num_slack_vars))
    nonbasic_vars = list(range(num_original_vars))

    # Compute initial solution
    starting_solution = [0] * num_original_vars + right_hand_side_values

    # Compute initial objective function coefficients
    table_objective_coefs = objective_coefs + [0] * num_slack_vars
    for i in basic_vars:
        for j, coef in enumerate(constraint_matrix[i - num_original_vars]):
            if i < num_original_vars:
                table_objective_coefs[j] -= coef * objective_coefs[i]

    # Minimization adjustment
    if optimization_goal == "min":
        table_objective_coefs = [-coef for coef in table_objective_coefs]

    # Compute constraint coefficients
    canonical_matrix = []
    for row in constraint_matrix:
        canonical_matrix.append(row)
    return canonical_matrix, right_hand_side_values, table_objective_coefs

## Алгоритм симплекс-метода:

1. Выбираем переменную, которую будем вводить в базис. Это делается в соответствии с указанным ранее принципом: мы должны выбрать переменную, возрастание которой приведет к росту функционала. Выбор происходит по следующему правилу:

Если задача на минимум – выбираем максимальный положительный элемент в последней строке.
Если задача на максимум – выбираем минимальный отрицательный.

Такой выбор, действительно, соответствует упомянутому выше принципу: если задача на минимум, то чем большее число вычитаем – тем быстрее убывает функционал; для максимума наоборот – чем большее число добавляем, тем быстрее функционал растет.

Замечание: Хотя мы и берем минимальное отрицательное число в задаче на максимум, этот коэффициент показывает направление роста функционала, т.к. строка функционала в симплекс-таблице взята со знаком “-”. Аналогичная ситуация с минимизацией.

Определение: Столбец симплекс-таблицы, отвечающий выбранному коэффициенту, называется ведущим столбцом.

In [42]:
# Determine the pivot position for the table
def determine_pivot_position(table):
    last_row = table[-1]
    column_idx = next(idx for idx, val in enumerate(last_row[:-1]) if val > 0)
    # Compute the ratios to determine the pivot row
    restrictions = [math.inf if eq[column_idx] <= 0 else eq[-1] / eq[column_idx] for eq in table[:-1]]
    row_idx = restrictions.index(min(restrictions))
    return row_idx, column_idx

2. Выбираем переменную, которую будем вводить в базис. Для этого нужно определить, какая из базисных переменных быстрее всего обратится в нуль при росте новой базисной переменной. Алгебраически это делается так:

Вектор правых частей почленно делится на ведущий столбец
Среди полученных значений выбирают минимальное положительное (отрицательные и нулевые ответы не рассматривают)

Определение: Такая строка называется ведущей строкой и отвечает переменной, которую нужно вывести из базиса.

Замечание: Фактически, мы выражаем старые базисные переменные из каждого уравнения системы ограничений через остальные переменные и смотрим, в каком уравнении возрастание новой базисной переменной быстрее всего даст 0. Попадание в такую ситуацию означает, что мы «наткнулись» на новую вершину. Именно поэтому нулевые и отрицательные элементы не рассматриваются, т.к. получение такого результата означает, что выбор такой новой базисной переменной будет уводить нас из области, вне которой решений не существует.

3. Ищем элемент, стоящий на пересечении ведущих строки и столбца.

Определение: Такой элемент называется ведущим элементом.

4. Вместо исключаемой переменной в первом столбце (с названиями базисных переменных) записываем название переменной, которую мы вводим в базис.

5. Далее начинается процесс вычисления нового базисного решения. Он происходит с помощью метода Жордана-Гаусса.

Новая Ведущая строка = Старая ведущая строка / Ведущий элемент
Новая строка = Новая строка – Коэффициент строки в ведущем столбце * Новая Ведущая строка

Замечание: Преобразование такого вида направлено на введение выбранной переменной в базис, т.е. представление ведущего столбца в виде базисного вектора.


In [43]:
# Perform the pivoting operation
def perform_pivoting(table, pivot_pos):
    new_table = [[] for _ in table]
    row_idx, col_idx = pivot_pos
    pivot_element = table[row_idx][col_idx]
    new_table[row_idx] = np.array(table[row_idx]) / pivot_element
    for eq_idx, eq in enumerate(table):
        if eq_idx != row_idx:
            factor = np.array(new_table[row_idx]) * table[eq_idx][col_idx]
            new_table[eq_idx] = np.array(table[eq_idx]) - factor
    return new_table

6. После этого проверяем условие оптимальности. Если полученное решение неоптимально – повторяем весь процесс снова.

Условие оптимальности полученного решения:

Если задача на максимум – в строке функционала нет отрицательных коэффициентов (т.е. при любом изменении переменных значение итогового функционала расти не будет).
Если задача на минимум – в строке функционала нет положительных коэффициентов (т.е. при любом изменении переменных значение итогового функционала уменьшаться не будет).

In [44]:
# Check if the solution can be improved
def solution_can_improve(table):
    last_row = table[-1]
    return any(val > 0 for val in last_row[:-1])

Неограниченность функционала

Однако, стоит отметить, что заданный функционал может не и достигать максимума/минимума в заданной области. Алгебраический признак этого можно сформулировать следующим образом:

При выборе ведущей строки (исключаемой переменной) результат почленного деления вектора правых частей на ведущий столбец содержит только нулевые и отрицательные значения.

Фактически, это значит, что какой бы рост мы не задавали новой базисной переменной, мы никогда не найдем новую вершину. А значит, наша функция не ограничена на множестве допустимых решений.

In [45]:
# Check if the solution is unbounded
def solution_is_unbounded(table):
    last_row = table[-1]
    column_idx = next((idx for idx, val in enumerate(last_row[:-1]) if val > 0), None)
    if column_idx is None:
        return False
    col_values = [row[column_idx] for row in table[:-1]]
    return all(val <= 0 for val in col_values)

Альтернативные решения

При нахождении оптимального решения возможен еще один вариант – есть альтернативные решения (другая угловая точка, дающая то же самое значение функционала).

Алгебраический признак существования альтернативы:

После достижения оптимального решения имеются нулевые коэффициенты при свободных переменных в строке функционала.

Это значит, что при росте соответствующей переменной с нулевым коэффициентом значение функционала не изменится и новое базисное решение будет также давать оптимум функционала.

In [46]:
# Check if a column is a basic column
def is_basic_column(column):
    return sum(column) == 1 and len([val for val in column if val == 0]) == len(column) - 1

In [47]:
# Check if there are alternate optimal solutions
def has_alternate_solutions(table):
    last_row = table[-1]
    for idx, val in enumerate(last_row[:-1]):
        if val == 0 and not is_basic_column(np.array(table)[:, idx]):
            return True
    return False

# Реализация симплекс метода

In [48]:
# Create the initial table
def create_table(table_objective_coefs, canonical_matrix, right_hand_side_values):
    slack_vars_added = [eq + [x] for eq, x in zip(canonical_matrix, right_hand_side_values)]
    z_row = table_objective_coefs + [0]
    return slack_vars_added + [z_row]

In [49]:
# Run the simplex method algorithm
def execute_simplex_method(table_objective_coefs, canonical_matrix, right_hand_side_values):
    table = create_table(table_objective_coefs, canonical_matrix, right_hand_side_values)
    # Iterate until the solution cannot be improved
    while solution_can_improve(table):
        if solution_is_unbounded(table):
            return "Unbounded"
        pivot_pos = determine_pivot_position(table)
        table = perform_pivoting(table, pivot_pos)
    # Extract the optimal solution or return infeasible
    return extract_solution(table) if not solution_is_unbounded(table) else "Infeasible"

In [50]:
# Extract the optimal solution from the table
def extract_solution(table):
    columns_transposed = np.array(table).T
    solutions = []
    for column in columns_transposed[:-1]:
        solution_val = 0
        if is_basic_column(column):
            idx_of_one = column.tolist().index(1)
            solution_val = columns_transposed[-1][idx_of_one]
        solutions.append(solution_val)
    return solutions

In [51]:
# Compute the value of the objective function
def compute_objective_value(solution, objective_coefs):
    return np.dot(solution, objective_coefs)

In [52]:
# Load the data and run the simplex method
data_content = load_json_data("test.json")
constraint_matrix, right_hand_side_values = transform_to_canonical_form(data_content)
objective_coefs = data_content["f"]
optimization_goal = data_content["goal"]
canonical_matrix, right_hand_side_values, table_objective_coefs = setup_initial_simplex_table(constraint_matrix, right_hand_side_values, objective_coefs, optimization_goal)
optimal_solution = execute_simplex_method(table_objective_coefs, canonical_matrix, right_hand_side_values)

# Print the results
if optimal_solution == "Unbounded":
    print("The problem has an unbounded solution.")
elif optimal_solution == "Infeasible":
    print("The problem has no feasible solutions.")
else:
    if has_alternate_solutions(create_table(table_objective_coefs, canonical_matrix, right_hand_side_values)):
        print('The problem has multiple optimal solutions.')
    print('Solution: ', optimal_solution)
    value = compute_objective_value(optimal_solution, table_objective_coefs)
    print('Value of the objective function: ', value)


The problem has multiple optimal solutions.
Solution:  [0, 2.0, 1.0, 1.0, 0]
Value of the objective function:  7.0


## Пример задачи

ЗАДАНИЕ. Компания производит полки для ванных комнат двух размеров – А и В. Агенты по
продаже считают, что в неделю на рынке может быть реализовано до 550 полок. Для каждой
полки типа А требуется 2 м2
материала, а для полки типа В – 3 м2
материала. Компания
может получить до 1200 м2
материала в неделю. Для изготовления одной полки типа А
требуется 12 мин машинного времени, а для изготовления одной полки типа В – 30 мин;
машину можно использовать 160 час в неделю. Если прибыль от продажи полок типа А
составляет 3 денежных единицы, а от полок типа В – 4 ден. ед., то сколько полок каждого
типа следует выпускать в неделю?
РЕШЕНИЕ. Составим математическую модель задачи. Пусть х1 – количество полок вида А, х2
– количество полок вида В, которые производятся в неделю (по смыслу задачи эти
переменные неотрицательны). Прибыль от продажи такого количества полок составит 3х1 +
4х2, прибыль требуется максимизировать. Выпишем ограничения задачи.
х1 + х2 ≤ 550 – в неделю на рынке может быть реализовано до 550 полок.
Затраты материала: 2х1 + 3х2 ≤ 1200
Затраты машинного времени: 12х1 + 30х2 ≤ 9600. 

Таким образом, приходим к задаче линейного программирования.
f = 3х1 + 4х2 → max,
х1 + х2 ≤ 550,
2х1 + 3х2 ≤ 1200,
12х1 + 30х2 ≤ 9600,
x1≥ 0, x2≥0. 


In [53]:
# Load the data and run the simplex method
data_content = load_json_data("test2.json")
constraint_matrix, right_hand_side_values = transform_to_canonical_form(data_content)
objective_coefs = data_content["f"]
optimization_goal = data_content["goal"]
canonical_matrix, right_hand_side_values, table_objective_coefs = setup_initial_simplex_table(constraint_matrix, right_hand_side_values, objective_coefs, optimization_goal)
optimal_solution = execute_simplex_method(table_objective_coefs, canonical_matrix, right_hand_side_values)

# Print the results
if optimal_solution == "Unbounded":
    print("The problem has an unbounded solution.")
elif optimal_solution == "Infeasible":
    print("The problem has no feasible solutions.")
else:
    if has_alternate_solutions(create_table(table_objective_coefs, canonical_matrix, right_hand_side_values)):
        print('The problem has multiple optimal solutions.')
    print('Solution: ', optimal_solution)
    value = compute_objective_value(optimal_solution, table_objective_coefs)
    print('Value of the objective function: ', value)

Solution:  [450.0, 100.0, 0, 0, 1200.0]
Value of the objective function:  1750.0


Целевая функция принимает значение 1750. Таким образом, чтобы получить максимальную прибыль, предприятию необходимо
производить 450 полок вида А и 100 полок вида В, при этом прибыль составит 1750 ден. ед.,
а останется неиспользованными 1200 минут (20 часов) машинного времени. 
